<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FPreprocessing/Preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install**

# **Import**

In [1]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time
import math
import random

from sklearn.model_selection import train_test_split

## Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation
!ls -al

/content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation
total 11032
drwx------ 2 root root     4096 Sep 14 08:50 'Case Report'
drwx------ 2 root root     4096 Aug 16 08:53  Dataset
-rw------- 1 root root    47630 Sep 23 11:31 '[Down Sampling] Surgical-Wound UNet.ipynb'
drwx------ 2 root root     4096 Sep 14 13:17  Lagacy
drwx------ 2 root root     4096 Aug 17 05:49  Log
-rw------- 1 root root    84314 Sep 23 17:09  Preprocessor.ipynb
drwx------ 2 root root     4096 Sep 14 06:04 'Raw Dataset'
-rw------- 1 root root 10999888 Sep 23 17:03 'Surgical-Wound UNet.ipynb'
-rw------- 1 root root   142540 Sep 23 04:36  Tester.ipynb


# **Grobal variable**

In [4]:
# Path (Absolute path in essential)
MODEL_PATH = "/content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation"

RAW_DATA_PATH = "/Raw Dataset"

WOUND_TRAIN_PATH = "/Wound/train"
WOUND_TEST_PATH = "/Wound/test"
CVC_INPUT_PATH = "/CVC-clinicDB/Original"
CVC_GT_PATH = "/CVC-clinicDB/Ground Truth"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Image preprocess
NORM_INPUT_W_SIZE = 224
NORM_INPUT_H_SIZE = 224

# **Function**

In [5]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

# **Make dir**

In [6]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

<br>
<br>
<br>
<br>
<br>
<br>

# ***Preprocess Performace Compare Test***


---

<br>

***공통사항***
* 원본 이미지 크기 = 224 * 224
* epoch = 100
* batch = 8
> UNet batch size = 4<br>
> Wound dataset batch size = 8
* lr = 1e-3
* Train:Val = 7:3
* Used model = Polar Res UNet++ (ISIC 2018 dice scroe rank no.1)

<br>

***주의사항***
* 이미지의 사이즈는 정규화 되어야 한다.
* Convolution에서 특징점을 잘 추출하기 때문에 색상영역이나, 스무딩/샤프닝 같은 필터 처리는 되려 성능저하 요소가 될 수 있다. 

<br>

***참고사항***
* CNN 대표 모델들의 네트워크 입력 사이즈는 224 * 224
* 이미지의 사이즈가 큰 경우에, Overlap-Tite(down sampling) 전략으로 한 이미지를 나눠서 학습시킨다.

<br>

***현재 결과***

---




<br>
<br>
<br>
<br>
<br>
<br>

## [***Control Group***](https://www.nature.com/articles/s41598-020-78799-w)
> [Train code](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/train.py)
> <br>
> [Data generator](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/utils/io/data.py#L10)
> <br>
> <br>
> Raw dataset에 대한 전처리 없이, load 후 model에 들어가기 전에 Normalize
> <br>
> * 현재까지 대조군 성능을 넘은 실험군이 없다.

```
model = Deeplabv3(input_shape=(input_dim_x, input_dim_y, 3), classes=1)
```

### Performance

#### **Color**

```
Input size =  torch.Size([8, 3, 224, 224])

TRAIN || EPOCH 0460 | BATCH  073 /  073 | DICE LOSS  0.0402
TRAIN || EPOCH 0461 | BATCH  073 /  073 | DICE LOSS  0.0424
TRAIN || EPOCH 0462 | BATCH  073 /  073 | DICE LOSS  0.0406
TRAIN || EPOCH 0463 | BATCH  073 /  073 | DICE LOSS  0.0432
TRAIN || EPOCH 0464 | BATCH  073 /  073 | DICE LOSS  0.0401

VALID || EPOCH 0460 | BATCH  032 /  032 | DICE LOSS  0.1069
VALID || EPOCH 0461 | BATCH  032 /  032 | DICE LOSS  0.1103
VALID || EPOCH 0462 | BATCH  032 /  032 | DICE LOSS  0.1140
VALID || EPOCH 0463 | BATCH  032 /  032 | DICE LOSS  0.1140
VALID || EPOCH 0464 | BATCH  032 /  032 | DICE LOSS  0.1146

Best loss in Train =  0.0392
Best loss in Validation =  0.0891
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1rG2arJcirP5erpfMvdt-vL3Gp6ArSlka"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1U43nlSo67GlVrQZZ7Gos74nxwTgvrjEY"  width = 1080>

### Arrange dataset
> Not change anything about input image(scar)

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    wound = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), wound)
    cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image)
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    wound = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), wound)
    cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image)
    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    wound = cv2.imread(gt)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), wound)
    cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image)
    idx += 1

<br>
<br>
<br>
<br>
<br>
<br>

## ***Experimental Group***

### Case 1

1. Delete Black Boundary(Padding)
2. Resize Wound Image to (224, 224)

#### **Color**


```

```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id="  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id="  width = 1080>

#### Preprocess

In [8]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    gt_image = cv2.resize(gt_image, 
                          (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    gt_image = cv2.resize(gt_image, 
                          (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    gt_image = cv2.resize(gt_image,
                          (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 2

1. Delete Black Boundary(Padding)
2. Pad the rest part to (224, 224) based on the middle of the image

#### **Color**


```
Input size =  torch.Size([8, 3, 224, 224])

TRAIN || EPOCH 0100 | BATCH  069 /  073 | DICE LOSS  0.1058
TRAIN || EPOCH 0100 | BATCH  070 /  073 | DICE LOSS  0.1062
TRAIN || EPOCH 0100 | BATCH  071 /  073 | DICE LOSS  0.1061
TRAIN || EPOCH 0100 | BATCH  072 /  073 | DICE LOSS  0.1057
TRAIN || EPOCH 0100 | BATCH  073 /  073 | DICE LOSS  0.1052

VALID || EPOCH 0100 | BATCH  028 /  032 | DICE LOSS  0.1309
VALID || EPOCH 0100 | BATCH  029 /  032 | DICE LOSS  0.1308
VALID || EPOCH 0100 | BATCH  030 /  032 | DICE LOSS  0.1356
VALID || EPOCH 0100 | BATCH  031 /  032 | DICE LOSS  0.1357
VALID || EPOCH 0100 | BATCH  032 /  032 | DICE LOSS  0.1401

Best loss in Train =  0.0920
Best loss in Validation =  0.1044
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1FqoLg0DMZwS0HtYXCfpn113oQvvU_Fic"  width = 320>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1j18BDF10_gfpgkYl_H0QZbPAdK4w1Px8"  width = 640>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])


    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 3

1. Delete Black Boundary(Padding)
2. Resize Wound Image to (224, 224) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) in the direction of right/down 

#### **Color**


```
Input size =  torch.Size([8, 3, 224, 224])

TRAIN || EPOCH 0100 | BATCH  069 /  073 | DICE LOSS  0.1401
TRAIN || EPOCH 0100 | BATCH  070 /  073 | DICE LOSS  0.1395
TRAIN || EPOCH 0100 | BATCH  071 /  073 | DICE LOSS  0.1397
TRAIN || EPOCH 0100 | BATCH  072 /  073 | DICE LOSS  0.1393
TRAIN || EPOCH 0100 | BATCH  073 /  073 | DICE LOSS  0.1425

VALID || EPOCH 0100 | BATCH  028 /  032 | DICE LOSS  0.1842
VALID || EPOCH 0100 | BATCH  029 /  032 | DICE LOSS  0.1921
VALID || EPOCH 0100 | BATCH  030 /  032 | DICE LOSS  0.1916
VALID || EPOCH 0100 | BATCH  031 /  032 | DICE LOSS  0.1882
VALID || EPOCH 0100 | BATCH  032 /  032 | DICE LOSS  0.1842

Best loss in Train =  0.1257
Best loss in Validation =  0.1525
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1Hz_tdEFXE9I8LqIfro3anAOw8RxbkjKS"  width = 320>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1ahoz3Wke5MKGqa55x9TEnejgElTM-5Pw"  width = 640>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access wound image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 4

1. Delete Black Boundary(Padding)
2. Resize Wound Image to (224, 224) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) based on the middle of the image

#### **Color**


```
Input size =  torch.Size([8, 3, 224, 224])

TRAIN || EPOCH 0100 | BATCH  069 /  073 | DICE LOSS  0.1341
TRAIN || EPOCH 0100 | BATCH  070 /  073 | DICE LOSS  0.1343
TRAIN || EPOCH 0100 | BATCH  071 /  073 | DICE LOSS  0.1336
TRAIN || EPOCH 0100 | BATCH  072 /  073 | DICE LOSS  0.1329
TRAIN || EPOCH 0100 | BATCH  073 /  073 | DICE LOSS  0.1329

VALID || EPOCH 0100 | BATCH  028 /  032 | DICE LOSS  0.1811
VALID || EPOCH 0100 | BATCH  029 /  032 | DICE LOSS  0.1797
VALID || EPOCH 0100 | BATCH  030 /  032 | DICE LOSS  0.1756
VALID || EPOCH 0100 | BATCH  031 /  032 | DICE LOSS  0.1734
VALID || EPOCH 0100 | BATCH  032 /  032 | DICE LOSS  0.1747

Best loss in Train =  0.1261
Best loss in Validation =  0.1446
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1ldhepTNZHrtIerQfm4jjEa6SIph9xo-X"  width = 320>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1-doK7qAEUSmic6YweHXnI5s2jXGcglBO"  width = 640>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    if resize_criteria == 0: # criteria is width
        pad_height = round((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    # Calculate padding size
    elif resize_criteria == 1: # criteria is heigth
        pad_width = round((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Print resize error
    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    if resize_criteria == 0: # criteria is width
        pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    # Calculate padding size
    elif resize_criteria == 1: # criteria is heigth
        pad_width = int((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Print resize error
    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access wound image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    if resize_criteria == 0: # criteria is width
        pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    # Calculate padding size
    elif resize_criteria == 1: # criteria is heigth
        pad_width = int((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Print resize error
    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 5

1. Delete Black Boundary(Padding)
2. Resize Wound Image to (204, 204) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) based on the middle of the image

#### **Color**


```
Input size =  torch.Size([8, 3, 224, 224])

TRAIN || EPOCH 0100 | BATCH  069 /  073 | DICE LOSS  0.1232
TRAIN || EPOCH 0100 | BATCH  070 /  073 | DICE LOSS  0.1237
TRAIN || EPOCH 0100 | BATCH  071 /  073 | DICE LOSS  0.1236
TRAIN || EPOCH 0100 | BATCH  072 /  073 | DICE LOSS  0.1232
TRAIN || EPOCH 0100 | BATCH  073 /  073 | DICE LOSS  0.1225

VALID || EPOCH 0100 | BATCH  028 /  032 | DICE LOSS  0.1741
VALID || EPOCH 0100 | BATCH  029 /  032 | DICE LOSS  0.1717
VALID || EPOCH 0100 | BATCH  030 /  032 | DICE LOSS  0.1719
VALID || EPOCH 0100 | BATCH  031 /  032 | DICE LOSS  0.1738
VALID || EPOCH 0100 | BATCH  032 /  032 | DICE LOSS  0.1694

Best loss in Train =  0.1165
Best loss in Validation =  0.1489
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=18o_acrSOCWL5oO-Q4haKxhJ-8cwsdgtr"  width = 320>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1vaB5sZaRGzBRUJX62Sbi9icL-pggkaxu"  width = 640>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (204, 204) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = (NORM_INPUT_W_SIZE - 20) / width
    resize_scale_h = (NORM_INPUT_H_SIZE - 20) / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)

    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    pad_height = round((NORM_INPUT_H_SIZE - height) / 2)
    pad_width = round((NORM_INPUT_W_SIZE - width) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width
        
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)

    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (204, 204) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = (NORM_INPUT_W_SIZE - 20) / width
    resize_scale_h = (NORM_INPUT_H_SIZE - 20) / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)
    
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    pad_height = round((NORM_INPUT_H_SIZE - height) / 2)
    pad_width = round((NORM_INPUT_W_SIZE - width) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width
        
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)

    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (204, 204) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = (NORM_INPUT_W_SIZE - 20) / width
    resize_scale_h = (NORM_INPUT_H_SIZE - 20) / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)
    
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    pad_height = round((NORM_INPUT_H_SIZE - height) / 2)
    pad_width = round((NORM_INPUT_W_SIZE - width) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width
        
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)

    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

# BACKUP

In [ ]:
def get_divisors(n):
    data = []

    divisor = 2
    while(divisor <= n):
        if n % divisor == 0:
            data.append(divisor)

        divisor = divisor * 2
            
    return data

# Init image index
w_idx = 0
gt_idx = 0

# Easy Max Adaptive Sliding Window
    # 1. feature_size의 약수 중 2의 배수를 계산하여 divisors에 저장한다.
    # 2. divisors 안의 숫자 중 가장 큰수를 골라 stride라고 정의한다.
    # 3. 기준 축(refer_axis)이 아닌 축의 길이를 slide_size라고 정의한다.
    # 4. (slide_size-feature_size) // stride를 계산하여 step_num으로 정의한다.
    # 5. (slide_size-feature_size) % stride를 계산하여 remain_pixel으로 정의한다.
    # 6. remain_pixel // step_num을 계산하여 extra_stride로 정의한다.
    # 7. remain_pixel % step_num을 계산하여 extra_random_stride로 정의한다.
    # 8. stride = stride + extra_stride
    # 9. extra_random_stride != 0이라면, 각 stride에 맞게 slide window를 진행하면서 랜덤하게 적어도 1번은 해당 step의 stride에 extra_random_stride를 더한다.
    # 10. window ROI의 (전체 픽셀-noneZeroCount)를 비교하여 gt의 흰색 픽셀 비율이 5% 미만이면 건너뛴다.

# 0.
feature_size = 224

# 1.
divisors = get_divisors(224)

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w > resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)
    
    # Calc white pixel in ground-truth for check proposed data
    gt_pixel_num = cv2.countNonZero(gt_image)
    if gt_pixel_num == 0: continue

    # Get height and width
    height, width = gt_image.shape

    # 2.
    stride = max(divisors)

    # 3.
    if height > width:
        assert width == feature_size, "Preprocessed image width is wrong, train[" + str(idx) + "] width is " + str(width)

        refer_axis = 'x'
        slide_size = height
    else:
        assert height == feature_size, "Preprocessed image width is height, train[" + str(idx) + "] height is " + str(height)
            
        refer_axis = 'y'
        slide_size = width

    # 4.
    step_num = (slide_size - feature_size) // stride
    if step_num == 0:
        cv2.imwrite(os.path.join(train_dir, f'wound_{w_idx:04d}.png'), image)
        cv2.imwrite(os.path.join(train_dir, f'gt_{gt_idx:04d}.png'), gt_image)

        w_idx += 1
        gt_idx += 1

        continue

    # 5.
    remain_pixel = (slide_size - feature_size) % stride

    # 6.
    extra_stride = remain_pixel // (step_num + 1)

    # 7.
    extra_random_stride = remain_pixel % (step_num + 1)
    extra_idx = random.randrange(0, (step_num + 1))

    # 8.
    stride += extra_stride

    # 9.
    images = []
    gt_images = []
    for step in range(0, step_num+1):
        tmp_stride = stride

        if extra_random_stride > 0 and step == extra_idx:
            tmp_stride += extra_random_stride
            extra_random_stride = -1 * extra_random_stride

        start_idx = tmp_stride * step
        end_idx = (tmp_stride * step) + feature_size
        if extra_random_stride < 0 and step != extra_idx:
            start_idx += extra_random_stride * (-1)
            end_idx += extra_random_stride * (-1)

        if refer_axis == 'x':
            slide_gt = gt_image[start_idx:end_idx, :]
            cnt_white_px = cv2.countNonZero(slide_gt)

            # 10.
            if cnt_white_px / gt_pixel_num > 0.05:
                images.append(image[start_idx:end_idx, :, :])
                gt_images.append(slide_gt)

        elif refer_axis == 'y':
            slide_gt = gt_image[:, start_idx:end_idx]
            cnt_white_px = cv2.countNonZero(slide_gt)

            # 10.
            if cnt_white_px / gt_pixel_num > 0.05:
                images.append(image[:, start_idx:end_idx, :])
                gt_images.append(slide_gt)

        else:
            assert True, "CODE ERROR, choice reference axis"

    # Save normalized image
    for wound in images:
        cv2.imwrite(os.path.join(train_dir, f'wound_{w_idx:04d}.png'), wound)
        w_idx += 1

    for ground_truth in gt_images:
        cv2.imwrite(os.path.join(train_dir, f'gt_{gt_idx:04d}.png'), ground_truth)
        gt_idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 6

1. Delete Black Boundary(Padding)
2. Resize Wound Image to 224 based on the smaller size(row or column), keeping the proportion
3. Sampling by moving the kernel both on input and ground-truth

#### **Color**


```

```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1ldhepTNZHrtIerQfm4jjEa6SIph9xo-X"  width = 320>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1-doK7qAEUSmic6YweHXnI5s2jXGcglBO"  width = 640>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w > resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image)
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w > resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image)
    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w > resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image)
    idx += 1